In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet101
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, Flatten, Input, Concatenate
import pickle
from sklearn.preprocessing import OneHotEncoder
from keras import Sequential
from keras.applications.xception import Xception
from keras.layers import BatchNormalization, Activation, Dropout, MaxPooling2D,Conv2D
from tensorflow.keras.layers.experimental import RandomFourierFeatures
import time
import os
from tensorflow_addons.metrics import F1Score

c:\Users\cdr03\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Import Training and test data

In [3]:
size = 224
num_classes = 16
NUM_EPOCHS = 10

In [4]:
from keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

auc_metric = tf.keras.metrics.AUC(
num_thresholds=200, curve="ROC",
summation_method="interpolation",
thresholds=None, multi_label=False
)
f1_score = F1Score(num_classes=6)

In [5]:
regularizer = tf.keras.regularizers.l1_l2(0, 0.01)

Change_input_layer, change the input layer to have only 1 channel (black and white). It sets the average of the pre-trained weights for the first layer and for the rest of the layers it uses the same weights.

In [6]:
def change_input_layer(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    biases  = layer.get_weights()[1]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel, biases])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model


Simple CNN

In [77]:
# Suggested by tensorflow
simplecnn = Sequential()
simplecnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 1)))
simplecnn.add(MaxPooling2D((2, 2)))
simplecnn.add(Conv2D(64, (3, 3), activation='relu'))
simplecnn.add(MaxPooling2D((2, 2)))
simplecnn.add(Conv2D(64, (3, 3), activation='relu'))
simplecnn.add(Flatten())
simplecnn.add(Dense(units=512, kernel_regularizer=regularizer))
simplecnn.add(Dense(units=1024, kernel_regularizer=regularizer))
simplecnn.add(BatchNormalization(scale=False,
                                   center=False))
simplecnn.add(Activation('relu'))
simplecnn.add(Dropout(rate=0.6))
simplecnn.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 


VGG16

In [7]:
vgg16_model = VGG16(include_top=False, weights='imagenet')
vgg16_updated = change_input_layer(vgg16_model, 1, size, size, 1)

['input_1', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 96, 96, 64)        640       
                                                                 
 block1_conv2 (Conv2D)       (None, 96, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 48, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (

In [8]:
final_vgg16 = Sequential()
final_vgg16.add(vgg16_updated)
final_vgg16.add(Flatten())
final_vgg16.add(Dense(units=1024, kernel_regularizer=regularizer))
final_vgg16.add(BatchNormalization(scale=False,
                                   center=False))
final_vgg16.add(Activation('relu'))
final_vgg16.add(Dropout(rate=0.6))
final_vgg16.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 


VGG19

In [9]:
https://github.com/bnsreenu/python_for_microscopists

SyntaxError: invalid syntax (214287814.py, line 1)

In [10]:
pretrained_model = VGG19(include_top=False, weights='imagenet')


In [11]:
vgg19_model = VGG19(include_top=False, weights='imagenet')
vgg19_updated = change_input_layer(vgg19_model, 1, size, size, 1)

['input_3', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_conv4', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_conv4', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_conv4', 'block5_pool']
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 96, 96, 64)        640       
                                                                 
 block1_conv2 (Conv2D)       (None, 96, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 48, 48, 64)        0         
                                                

In [12]:
final_vgg19 = Sequential()
final_vgg19.add(vgg19_updated)
final_vgg19.add(Flatten())
final_vgg19.add(Dense(units=512, kernel_regularizer=regularizer))
final_vgg19.add(Dense(units=1024, kernel_regularizer=regularizer))
final_vgg19.add(BatchNormalization(scale=False,
                                   center=False))
final_vgg19.add(Activation('relu'))
final_vgg19.add(Dropout(rate=0.6))
final_vgg19.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 

Resnet

In [13]:
resnet_model = ResNet101(include_top=False, weights='imagenet')
resnet_model.summary()
resnet_updated = change_input_layer(resnet_model, 2, size, size, 1)
resnet_updated.trainable = True

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_4[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                       

In [14]:
resnet_updated.trainable = False
final_resnet = Sequential()
final_resnet.add(resnet_updated)
final_resnet.add(Flatten())
final_resnet.add(Dense(units=1024, kernel_regularizer=regularizer))
final_resnet.add(BatchNormalization(scale=False,
                                   center=False))
final_resnet.add(Activation('relu'))
final_resnet.add(Dropout(rate=0.6))
final_resnet.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 

DenseNet

In [9]:
def change_input_layer_dense(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model

In [16]:
densenet_model = DenseNet121(include_top=False, weights='imagenet')
densenet_updated = change_input_layer_dense(densenet_model, 2, size, size, 1)

['input_5', 'zero_padding2d', 'conv1/conv', 'conv1/bn', 'conv1/relu', 'zero_padding2d_1', 'pool1', 'conv2_block1_0_bn', 'conv2_block1_0_relu', 'conv2_block1_1_conv', 'conv2_block1_1_bn', 'conv2_block1_1_relu', 'conv2_block1_2_conv', 'conv2_block1_concat', 'conv2_block2_0_bn', 'conv2_block2_0_relu', 'conv2_block2_1_conv', 'conv2_block2_1_bn', 'conv2_block2_1_relu', 'conv2_block2_2_conv', 'conv2_block2_concat', 'conv2_block3_0_bn', 'conv2_block3_0_relu', 'conv2_block3_1_conv', 'conv2_block3_1_bn', 'conv2_block3_1_relu', 'conv2_block3_2_conv', 'conv2_block3_concat', 'conv2_block4_0_bn', 'conv2_block4_0_relu', 'conv2_block4_1_conv', 'conv2_block4_1_bn', 'conv2_block4_1_relu', 'conv2_block4_2_conv', 'conv2_block4_concat', 'conv2_block5_0_bn', 'conv2_block5_0_relu', 'conv2_block5_1_conv', 'conv2_block5_1_bn', 'conv2_block5_1_relu', 'conv2_block5_2_conv', 'conv2_block5_concat', 'conv2_block6_0_bn', 'conv2_block6_0_relu', 'conv2_block6_1_conv', 'conv2_block6_1_bn', 'conv2_block6_1_relu', 'conv

In [17]:
final_densenet = Sequential()
final_densenet.add(densenet_updated)
final_densenet.add(Flatten())
final_densenet.add(Dense(units=512, kernel_regularizer=regularizer))
final_densenet.add(Dense(units=1024, kernel_regularizer=regularizer))
final_densenet.add(BatchNormalization(scale=False,
                                   center=False))
final_densenet.add(Activation('relu'))
final_densenet.add(Dropout(rate=0.6))
final_densenet.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 

Xception

In [10]:
size = 224
xception_model = Xception(include_top=False, weights='imagenet')
xception_model.summary()
xception_updated = change_input_layer_dense(xception_model, 1, size, size, 1)

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                         32)                                                        

In [11]:
final_xception = Sequential()
final_xception.add(xception_updated)
final_xception.add(Flatten())
final_xception.add(Dense(units=1024, kernel_regularizer=regularizer))
final_xception.add(BatchNormalization(scale=False,
                                   center=False))
final_xception.add(Activation('relu'))
final_xception.add(Dropout(rate=0.6))
final_xception.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 

Training Emotion

In [13]:
import pandas as pd
import numpy as np

CK+

In [21]:
CK_Emotion_X_train = pd.read_pickle("./Clean_datasets/CK+/CK+_Emotion_X_train")
CK_Emotion_y_train = pd.read_pickle("./Clean_datasets/CK+/CK+_Emotion_y_train")
CK_Emotion_X_val = pd.read_pickle("./Clean_datasets/CK+/CK+_Emotion_X_val")
CK_Emotion_y_val = pd.read_pickle("./Clean_datasets/CK+/CK+_Emotion_y_val")

In [14]:
CK_FACS_X_train = pd.read_pickle("./Clean_datasets/CK+/CK+_FACS_X_train")
CK_FACS_y_train = pd.read_pickle("./Clean_datasets/CK+/CK+_FACS_y_train")
CK_FACS_X_val = pd.read_pickle("./Clean_datasets/CK+/CK+_FACS_X_val")
CK_FACS_y_val = pd.read_pickle("./Clean_datasets/CK+/CK+_FACS_y_val")

In [15]:
X_train = CK_FACS_X_train
X_val = CK_FACS_X_val
y_train = CK_FACS_y_train
y_val = CK_FACS_y_val

In [28]:
y_train = CK_FACS_y_train
y_val = CK_FACS_y_val

In [22]:
X_train = CK_Emotion_X_train
X_val = CK_Emotion_X_val
y_train = CK_Emotion_y_train
y_val = CK_Emotion_y_val

In [22]:
dataset_name = "CK+"

EmotioNet

In [55]:
EmotioNet_Emotion_X_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_train")
EmotioNet_Emotion_y_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_y_train")
EmotioNet_Emotion_X_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_X_val")
EmotioNet_Emotion_y_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_Emotion_y_val")

In [101]:
X_train = EmotioNet_Emotion_X_train
X_val = EmotioNet_Emotion_X_val
y_train = EmotioNet_Emotion_y_train
y_val = EmotioNet_Emotion_y_val

NameError: name 'EmotioNet_Emotion_X_train' is not defined

In [66]:
dataset_name = "EmotioNet"

MMI

In [80]:
MMI_Emotion_X_train = pd.read_pickle("./Clean_datasets/MMI/MMI_Emotion_X_train")
MMI_Emotion_y_train = pd.read_pickle("./Clean_datasets/MMI/MMI_Emotion_y_train")
MMI_Emotion_X_val = pd.read_pickle("./Clean_datasets/MMI/MMI_Emotion_X_val")
MMI_Emotion_y_val = pd.read_pickle("./Clean_datasets/MMI/MMI_Emotion_y_val")

In [81]:
X_train = MMI_Emotion_X_train
X_val = MMI_Emotion_X_val
y_train = MMI_Emotion_y_train
y_val = MMI_Emotion_y_val

In [82]:
dataset_name = "MMI"

FER2013

In [132]:
FER2013_Emotion_X_train = pd.read_pickle("./Clean_datasets/FER2013/FER2013_Emotion_X_train")
FER2013_Emotion_y_train = pd.read_pickle("./Clean_datasets/FER2013/FER2013_Emotion_Y_train")
FER2013_Emotion_X_val = pd.read_pickle("./Clean_datasets/FER2013/FER2013_Emotion_X_val")
FER2013_Emotion_y_val = pd.read_pickle("./Clean_datasets/FER2013/FER2013_Emotion_Y_val")

In [133]:
X_train = FER2013_Emotion_X_train
X_val = FER2013_Emotion_X_val
y_train = FER2013_Emotion_y_train
y_val = FER2013_Emotion_y_val

In [134]:
dataset_name = "FER2013"

Data pre-processing to fit the models

In [81]:
import numpy as np
from skimage.feature import hog

def extract_hog_features(images):
    features = []
    for image in images:
        fd, hog_image = hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
        features.append(fd)
    return np.array(features)

In [82]:
X_train_hog = extract_hog_features(X_train)
X_val_hog = extract_hog_features(X_val)


ValueError: Only images with two spatial dimensions are supported. If using with color/multichannel images, specify `channel_axis`.

In [16]:
X_train = [list(i) for i in X_train.values]
#X_train = tf.convert_to_tensor(X_train)
X_val = [list(i) for i in X_val.values]
#X_val = tf.convert_to_tensor(X_val)

In [17]:
X_train = np.asarray(X_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')

In [18]:
X_train = np.resize(X_train,(len(X_train),size,size,1))
X_val = np.resize(X_val,(len(X_val),size,size,1))

In [19]:
print(np.shape(X_train))
print(np.shape(X_val))

(468, 224, 224, 1)
(54, 224, 224, 1)


In [68]:
y_train

,1,2,4,5,6,9,10,12,15,17,18,20,24,25,26,28
0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0
1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
4,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
585,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
586,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
587,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [110]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
y_train = np.array(np.array(y_train).reshape(-1, 1))
y_val = np.array(np.array(y_val).reshape(-1, 1))

In [33]:
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder.fit_transform(y_train)
onehot_encoder2 = OneHotEncoder(sparse_output=False)
y_val = onehot_encoder2.fit_transform(y_val)

In [111]:
np.shape(y_val)

(54, 16)

In [112]:
np.shape(y_train)

(468, 16)

In [36]:
NUM_EPOCHS = 20

SVM

In [80]:
SVM = keras.Sequential(
    [
        keras.Input(shape=(X_train_hog.shape[1],)),
        RandomFourierFeatures(
            output_dim=4096, scale=10.0, kernel_initializer="gaussian"
        ),
        Dense(units=16),
    ]
)
SVM.compile(
    optimizer='adam',
    loss=keras.losses.hinge,
    metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])


history_SVM = SVM.fit(X_train_hog,y_train,validation_data=(X_val_hog,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

ValueError: Data cardinality is ambiguous:
  x sizes: 247
  y sizes: 468
Make sure all arrays contain the same number of samples.

Simple CNN

In [56]:
import datetime
folder_name = 'logs/SimpleCNN'+dataset_name
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [58]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/SimpleCNN'+dataset_name,monitor='val_f1_m', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [78]:
simplecnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

In [73]:
simplecnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

In [79]:
history_simplecnn = simplecnn.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

Epoch 1/10
15/15 [==============================] - ETA: 0s - loss: 11.3096 - precision_8: 0.4048 - recall_7: 0.0246 - custom_f1: 0.0458 - auc_4: 0.5929 - accuracy: 0.0876WARNING:tensorflow:Can save best model only with val_f1_m available, skipping.


15/15 [==============================] - 9s 468ms/step - loss: 11.3096 - precision_8: 0.4048 - recall_7: 0.0246 - custom_f1: 0.0458 - auc_4: 0.5929 - accuracy: 0.0876 - val_loss: 6.1075 - val_precision_8: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_4: 0.6544 - val_accuracy: 0.0000e+00
Epoch 2/10
15/15 [==============================] - ETA: 0s - loss: 4.4775 - precision_8: 0.5238 - recall_7: 0.0239 - custom_f1: 0.0448 - auc_4: 0.6783 - accuracy: 0.1218WARNING:tensorflow:Can save best model only with val_f1_m available, skipping.


15/15 [==============================] - 7s 459ms/step - loss: 4.4775 - precision_8: 0.5238 - recall_7: 0.0239 - custom_f1: 0.0448 - auc_4: 0.6783 - accuracy: 0.1218 - val_loss: 3.3238 - val_precision_8: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_4: 0.7120 - val_accuracy: 0.0000e+00
Epoch 3/10
15/15 [==============================] - ETA: 0s - loss: 2.4675 - precision_8: 0.7308 - recall_7: 0.0137 - custom_f1: 0.0270 - auc_4: 0.7244 - accuracy: 0.1282WARNING:tensorflow:Can save best model only with val_f1_m available, skipping.


15/15 [==============================] - 7s 435ms/step - loss: 2.4675 - precision_8: 0.7308 - recall_7: 0.0137 - custom_f1: 0.0270 - auc_4: 0.7244 - accuracy: 0.1282 - val_loss: 2.0123 - val_precision_8: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_4: 0.7136 - val_accuracy: 0.0741
Epoch 4/10
15/15 [==============================] - ETA: 0s - loss: 1.5020 - precision_8: 0.6047 - recall_7: 0.0188 - custom_f1: 0.0375 - auc_4: 0.7280 - accuracy: 0.1325WARNING:tensorflow:Can save best model only with val_f1_m available, skipping.


15/15 [==============================] - 7s 453ms/step - loss: 1.5020 - precision_8: 0.6047 - recall_7: 0.0188 - custom_f1: 0.0375 - auc_4: 0.7280 - accuracy: 0.1325 - val_loss: 1.3725 - val_precision_8: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_4: 0.7361 - val_accuracy: 0.0741
Epoch 5/10
15/15 [==============================] - ETA: 0s - loss: 1.0385 - precision_8: 0.5577 - recall_7: 0.0210 - custom_f1: 0.0391 - auc_4: 0.7323 - accuracy: 0.1368WARNING:tensorflow:Can save best model only with val_f1_m available, skipping.


15/15 [==============================] - 7s 447ms/step - loss: 1.0385 - precision_8: 0.5577 - recall_7: 0.0210 - custom_f1: 0.0391 - auc_4: 0.7323 - accuracy: 0.1368 - val_loss: 1.0717 - val_precision_8: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_4: 0.7364 - val_accuracy: 0.0741
Epoch 6/10
 6/15 [===========>..................] - ETA: 3s - loss: 0.8435 - precision_8: 0.5000 - recall_7: 0.0070 - custom_f1: 0.0138 - auc_4: 0.7637 - accuracy: 0.1562

KeyboardInterrupt: 

VGG16

In [97]:
import datetime
folder_name = 'logs/VGG16'+dataset_name
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [98]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/VGG16'+dataset_name,monitor='val_f1_m', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [99]:
final_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

In [100]:
history_vgg16 = final_vgg16.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

6/6 [==============================] - ETA: 0s - loss: 3.0515 - precision_m: 0.1305 - recall_m: 0.0938 - f1_m: 0.1086 - accuracy: 0.2083

INFO:tensorflow:Assets written to: ./chkpts\VGG16MMI\assets


INFO:tensorflow:Assets written to: ./chkpts\VGG16MMI\assets


6/6 [==============================] - 14s 2s/step - loss: 3.0515 - precision_m: 0.1305 - recall_m: 0.0938 - f1_m: 0.1086 - accuracy: 0.2083 - val_loss: 3.9126 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258


VGG19

In [101]:
folder_name = 'logs/VGG19'+dataset_name
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [102]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/VGG19'+dataset_name,monitor='val_f1_m', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [103]:
final_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

NameError: name 'final_vgg19' is not defined

In [104]:
history_vgg19 = final_vgg19.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

NameError: name 'final_vgg19' is not defined

ResNet

In [ ]:
folder_name = 'logs/ResNet'+dataset_name
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [ ]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/ResNet'+dataset_name,monitor='val_f1_m', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [ ]:
final_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

In [ ]:
history_resnet = final_resnet.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

Epoch 1/20
28/28 [==============================] - ETA: 0s - loss: 2.2791 - precision_m: 0.5765 - recall_m: 0.5076 - f1_m: 0.5381 - accuracy: 0.5599

INFO:tensorflow:Assets written to: .\chkpts\assets


INFO:tensorflow:Assets written to: .\chkpts\assets


28/28 [==============================] - 43s 2s/step - loss: 2.2791 - precision_m: 0.5765 - recall_m: 0.5076 - f1_m: 0.5381 - accuracy: 0.5599 - val_loss: 13.7011 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258
Epoch 2/20
28/28 [==============================] - 39s 1s/step - loss: 1.4844 - precision_m: 0.7014 - recall_m: 0.5784 - f1_m: 0.6324 - accuracy: 0.6613 - val_loss: 9.1128 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258
Epoch 3/20
28/28 [==============================] - 41s 1s/step - loss: 1.3765 - precision_m: 0.7215 - recall_m: 0.6013 - f1_m: 0.6543 - accuracy: 0.6762 - val_loss: 6.5305 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258
Epoch 4/20
28/28 [==============================] - 40s 1s/step - loss: 1.2801 - precision_m: 0.7268 - recall_m: 0.6140 - f1_m: 0.6645 - accuracy: 0.6693 - val_loss: 5.5872 - val_precision_m: 0.2258 - val_recall_m: 0.2

KeyboardInterrupt: 

DenseNet

In [ ]:
folder_name = 'logs/DenseNet'+dataset_name
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [ ]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/DenseNet'+dataset_name,monitor='val_f1_m', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [ ]:
final_densenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

In [ ]:
history_densenet = final_densenet.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

Epoch 1/20
28/28 [==============================] - ETA: 0s - loss: 2.2791 - precision_m: 0.5765 - recall_m: 0.5076 - f1_m: 0.5381 - accuracy: 0.5599

INFO:tensorflow:Assets written to: .\chkpts\assets


INFO:tensorflow:Assets written to: .\chkpts\assets


28/28 [==============================] - 43s 2s/step - loss: 2.2791 - precision_m: 0.5765 - recall_m: 0.5076 - f1_m: 0.5381 - accuracy: 0.5599 - val_loss: 13.7011 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258
Epoch 2/20
28/28 [==============================] - 39s 1s/step - loss: 1.4844 - precision_m: 0.7014 - recall_m: 0.5784 - f1_m: 0.6324 - accuracy: 0.6613 - val_loss: 9.1128 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258
Epoch 3/20
28/28 [==============================] - 41s 1s/step - loss: 1.3765 - precision_m: 0.7215 - recall_m: 0.6013 - f1_m: 0.6543 - accuracy: 0.6762 - val_loss: 6.5305 - val_precision_m: 0.2258 - val_recall_m: 0.2258 - val_f1_m: 0.2258 - val_accuracy: 0.2258
Epoch 4/20
28/28 [==============================] - 40s 1s/step - loss: 1.2801 - precision_m: 0.7268 - recall_m: 0.6140 - f1_m: 0.6645 - accuracy: 0.6693 - val_loss: 5.5872 - val_precision_m: 0.2258 - val_recall_m: 0.2

KeyboardInterrupt: 

Xception

In [24]:
import datetime
folder_name = 'logs/Xception'+dataset_name
log_folder = folder_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [25]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Xception'+dataset_name,monitor='val_f1_m', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
callbacks=[model_checkpoint_cb, tensorboard_callback]

In [26]:
final_xception.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.Precision(),keras.metrics.Recall(),custom_f1, auc_metric,'accuracy'])

In [1]:
tf.config.run_functions_eagerly(False)

NameError: name 'tf' is not defined

In [30]:
history_xception = final_xception.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=NUM_EPOCHS,
callbacks=callbacks)

Epoch 1/10


KeyboardInterrupt: 